In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __init__ import *

In [ ]:
comparison_x_sample = pd.read_csv(
    "../output/comparison_x_sample.tsv", sep="\t", index_col=0
)

set_x_sample = pd.read_csv("../output/set_x_sample.tsv", sep="\t", index_col=0)

In [ ]:
name_to_sets = {}

for name in os.listdir("../input/feature_set/"):

    name_to_sets[name] = kraft.gmt.read("../input/feature_set/{}".format(name))

In [ ]:
# TODO: plot: x = "Score", y = "1 - P-Value"

keyword_arguments = {
    "n_job": SETTING["n_job"],
    "n_sampling": SETTING["n_sampling"],
    "n_permutation": SETTING["n_permutation"],
    "se_data_type": "binary",
    "plot_std": SETTING["plot_std"],
}

for comparison, groups in comparison_x_sample.iterrows():

    groups.dropna(inplace=True)

    min_group_size = np.unique(groups.to_numpy(), return_counts=True)[1].min()

    directory_path = os.path.join(
        "../output/find_differentially_expressed_set/", kraft.path.clean(comparison)
    )

    kraft.path.path(directory_path)

    for function, group_size in ((kraft.information.get_ic, 2),):

        if min_group_size < group_size:

            continue

        directory_path_ = os.path.join(
            directory_path, "all_{}/".format(function.__name__)
        )

        kraft.path.path(directory_path_)

        scores = kraft.function_heat_map.make(
            groups,
            set_x_sample,
            function,
            n_extreme=SETTING["n_extreme"],
            title="All ({})".format(function.__name__),
            directory_path=directory_path_,
            **keyword_arguments,
        )

        kraft.function_heat_map.make(
            groups,
            set_x_sample.reindex(index=SETTING["sets_to_peek"]),
            function,
            scores=scores,
            n_extreme=None,
            title="Peek ({})".format(function.__name__),
            **keyword_arguments,
        )

        for name, sets in name_to_sets.items():

            directory_path_ = os.path.join(directory_path, name, "")

            kraft.path.path(directory_path_)

            common_sets = set_x_sample.index & sets

            if 0 < len(common_sets):

                kraft.function_heat_map.make(
                    groups,
                    set_x_sample.loc[common_sets, :],
                    function,
                    scores=scores,
                    n_extreme=SETTING["n_extreme"],
                    title=name,
                    directory_path=directory_path_,
                    **keyword_arguments,
                )